In [124]:

# === CONFIGURACIÓN DE ESTILO INSTITUCIONAL CORHUILA ===
import seaborn as sns
import plotly.express as px
import plotly.io as pio
import json

COLORES_CORHUILA = {
    "verde": "#009739",
    "azul": "#005B7F",
    "blanco": "#FFFFFF",
    "negro": "#000000",
    "gris_claro": "#F0F0F0"
}

sns.set_palette([COLORES_CORHUILA['verde'], COLORES_CORHUILA['azul']])
px.defaults.template = "plotly_white"
px.defaults.color_discrete_sequence = [COLORES_CORHUILA['verde'], COLORES_CORHUILA['azul']]

# Cargar y registrar el template visual avanzado CORHUILA
with open('estilo_corhuila_plotly.json', 'r', encoding='utf-8') as f:
    corhuila_template = json.load(f)
pio.templates['corhuila'] = corhuila_template

# === FUNCIÓN DE LAYOUT INSTITUCIONAL PARA PLOTLY (MEJORADA) ===
from PIL import Image
import base64
from io import BytesIO

def corhuila_layout(fig, with_logo=True, width=700, height=400):
    fig.update_layout(
        template='corhuila',
        width=width,
        height=height,
        # Glassmorphism effect (tarjeta flotante)
        shapes=[dict(
            type='rect', xref='paper', yref='paper',
            x0=0, y0=0, x1=1, y1=1,
            fillcolor='rgba(255,255,255,0.55)',
            line=dict(color=COLORES_CORHUILA['verde'], width=4),
            layer='below',
            opacity=0.7,
            editable=False
        )],
        # Sombra sutil
        annotations=[dict(
            text='',
            showarrow=False,
            xref='paper', yref='paper',
            x=0.5, y=0.5,
            bgcolor='rgba(0,0,0,0.08)',
            opacity=0.3,
            borderpad=20
        )],
    )
    # Logo institucional (PNG a base64)
    if with_logo:
        try:
            with open("logo_corhuila.png", "rb") as image_file:
                encoded = base64.b64encode(image_file.read()).decode()
            fig.update_layout(images=[dict(
                source=f"data:image/png;base64,{encoded}",
                xref="paper", yref="paper",
                x=1, y=1,
                sizex=0.13, sizey=0.13,
                xanchor="right", yanchor="top",
                opacity=0.98,
                layer="above"
            )])
        except Exception:
            pass
    return fig


# Tablero Interactivo: Análisis de Matrícula Universitaria y Perfil Estudiantil

Este notebook guiará paso a paso la construcción del tablero, iniciando con la carga y exploración de los datos.

## 1. Carga y exploración inicial de datos

In [125]:
import pandas as pd

# Cargar el archivo Excel
df = pd.read_excel('Estudiantes_Universidad_2025_U3.xlsx')

# Vista previa de los datos
df.head()

,Nombre del Estudiante,Carrera,Costo por Crédito (USD),Costo de Materiales (USD),Fecha de Matrícula,Lugar de Procedencia,Costos Administrativos (USD),Número de Cursos Matriculados,Incremento de Matrícula 2025 (%),Número de Créditos Matriculados,Promedio Académico Anterior,Número de Semestres Cursados,Edad del Estudiante
0,Guillermo Negrón,Arquitectura,290,350,2024-10-12,Perú,190,6,5,5,4.09,9,23
1,Carolina Cordero,Administración de Empresas,240,210,2024-12-04,Perú,150,4,5,4,2.56,7,25
2,Marisela Rael,Ingeniería de Sistemas,260,250,2024-05-16,Perú,160,5,5,3,3.87,1,24
3,Alberto Timoteo,Derecho,250,200,2024-09-10,Perú,150,3,5,7,4.22,5,26
4,Jaqueline Guerrero,Ingeniería Electrónica,270,280,2024-12-18,Paraguay,170,6,5,8,4.01,7,18


In [126]:
# Información general del DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 13 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   Nombre del Estudiante             50 non-null     object        
 1   Carrera                           50 non-null     object        
 2   Costo por Crédito (USD)           50 non-null     int64         
 3   Costo de Materiales (USD)         50 non-null     int64         
 4   Fecha de Matrícula                50 non-null     datetime64[ns]
 5   Lugar de Procedencia              50 non-null     object        
 6   Costos Administrativos (USD)      50 non-null     int64         
 7   Número de Cursos Matriculados     50 non-null     int64         
 8   Incremento de Matrícula 2025 (%)  50 non-null     int64         
 9   Número de Créditos Matriculados   50 non-null     int64         
 10  Promedio Académico Anterior       50 non-null     fl

In [127]:
# Estadísticas descriptivas
df.describe()

,Costo por Crédito (USD),Costo de Materiales (USD),Fecha de Matrícula,Costos Administrativos (USD),Número de Cursos Matriculados,Incremento de Matrícula 2025 (%),Número de Créditos Matriculados,Promedio Académico Anterior,Número de Semestres Cursados,Edad del Estudiante
count,50.000000,50.000000,50,50.000000,50.000000,50.0,50.000000,50.00000,50.000000,50.000000
mean,249.000000,235.400000,2024-07-18 05:45:36,157.400000,5.420000,5.0,5.740000,3.77700,4.580000,23.840000
min,200.000000,160.000000,2024-01-19 00:00:00,130.000000,3.000000,5.0,2.000000,2.52000,1.000000,17.000000
25%,222.500000,182.500000,2024-04-27 00:00:00,141.250000,4.000000,5.0,5.000000,3.04500,3.000000,20.000000
50%,250.000000,210.000000,2024-06-25 00:00:00,150.000000,6.000000,5.0,6.000000,3.94500,5.000000,25.000000
75%,277.500000,295.000000,2024-10-25 12:00:00,177.500000,7.000000,5.0,7.000000,4.32750,6.750000,27.000000
max,290.000000,350.000000,2024-12-26 00:00:00,190.000000,8.000000,5.0,9.000000,4.98000,9.000000,30.000000
std,27.792966,58.491583,NaN,18.467843,1.819284,0.0,1.675879,0.74633,2.382676,4.032318


---

Continúa con la limpieza de datos y el análisis exploratorio en la siguiente sección.

## 2. Limpieza de datos

En esta sección se revisan valores nulos, tipos de datos y duplicados. Se documentan los hallazgos y las decisiones tomadas para dejar los datos listos para el análisis.

In [128]:
# Revisar valores nulos por columna
df.isnull().sum()

Nombre del Estudiante               0
Carrera                             0
Costo por Crédito (USD)             0
Costo de Materiales (USD)           0
Fecha de Matrícula                  0
Lugar de Procedencia                0
Costos Administrativos (USD)        0
Número de Cursos Matriculados       0
Incremento de Matrícula 2025 (%)    0
Número de Créditos Matriculados     0
Promedio Académico Anterior         0
Número de Semestres Cursados        0
Edad del Estudiante                 0
dtype: int64

In [129]:
# Revisar tipos de datos
df.dtypes

Nombre del Estudiante                       object
Carrera                                     object
Costo por Crédito (USD)                      int64
Costo de Materiales (USD)                    int64
Fecha de Matrícula                  datetime64[ns]
Lugar de Procedencia                        object
Costos Administrativos (USD)                 int64
Número de Cursos Matriculados                int64
Incremento de Matrícula 2025 (%)             int64
Número de Créditos Matriculados              int64
Promedio Académico Anterior                float64
Número de Semestres Cursados                 int64
Edad del Estudiante                          int64
dtype: object

In [130]:
# Eliminar duplicados si existen
df = df.drop_duplicates()

- Si se encuentran valores nulos, decidir si imputar (rellenar) o eliminar filas/columnas afectadas.
- Documentar cualquier cambio realizado en los datos.

¿Deseas que realice limpieza automática o prefieres revisar los resultados antes de modificar los datos?

## 3. Cálculo de KPIs Financieros y Demográficos

En esta sección se calculan los principales indicadores clave de desempeño (KPIs) solicitados en la actividad:
- Costo total de matrícula por estudiante
- Proyección con incremento del 5% para 2025
- Costo promedio por carrera
- Ingreso total estimado para la universidad
- Edad promedio por carrera

In [131]:
# Ajuste de nombres de columnas para el cálculo de KPIs
# Mostramos los nombres originales para referencia
print(df.columns.tolist())

# Renombrar columnas a nombres estándar para los KPIs
# (ajusta si tu archivo tiene nombres diferentes)
df = df.rename(columns={
    'Costo por Crédito (USD)': 'Costo_por_Credito',
    'Costo de Materiales (USD)': 'Costo_Materiales',
    'Costos Administrativos (USD)': 'Costos_Administrativos',
    'Número de Créditos Matriculados': 'Creditos_Matriculados',
    'Promedio Académico Anterior': 'Promedio_Academico',
    'Lugar de Procedencia': 'Pais_Procedencia',
    'Edad del Estudiante': 'Edad',
    'Carrera': 'Carrera',
    'Nombre del Estudiante': 'Nombre',
    'Fecha de Matrícula': 'Fecha_Matricula',
    # Agrega aquí otros ajustes si es necesario
})

['Nombre del Estudiante', 'Carrera', 'Costo por Crédito (USD)', 'Costo de Materiales (USD)', 'Fecha de Matrícula', 'Lugar de Procedencia', 'Costos Administrativos (USD)', 'Número de Cursos Matriculados', 'Incremento de Matrícula 2025 (%)', 'Número de Créditos Matriculados', 'Promedio Académico Anterior', 'Número de Semestres Cursados', 'Edad del Estudiante']


In [132]:
# Ahora sí, cálculo de KPIs con los nombres corregidos
# Costo total de matrícula por estudiante
# (Costo por Crédito × Créditos Matriculados) + Costo de Materiales + Costos Administrativos
df['Costo_Total'] = (df['Costo_por_Credito'] * df['Creditos_Matriculados']) + df['Costo_Materiales'] + df['Costos_Administrativos']

# Proyección con incremento del 5% para 2025
df['Costo_Proyeccion_2025'] = df['Costo_Total'] * 1.05

# Costo promedio por carrera
costo_promedio_carrera = df.groupby('Carrera')['Costo_Total'].mean().sort_values(ascending=False)
print('Costo promedio por carrera:')
print(costo_promedio_carrera)

# Ingreso total estimado para la universidad
ingreso_total = df['Costo_Total'].sum()
print(f"Ingreso total estimado para la universidad: ${ingreso_total:,.2f}")

# Edad promedio por carrera
edad_promedio_carrera = df.groupby('Carrera')['Edad'].mean().sort_values(ascending=False)
print('Edad promedio por carrera:')
print(edad_promedio_carrera)

Costo promedio por carrera:
Carrera
Ingeniería Electrónica         2272.500000
Arquitectura                   2207.500000
Ingeniería Industrial          2097.777778
Ingeniería de Sistemas         2022.000000
Derecho                        1900.000000
Contaduría                     1830.000000
Administración de Empresas     1680.000000
Licenciatura en Matemáticas    1490.000000
Psicología                     1310.000000
Name: Costo_Total, dtype: float64
Ingreso total estimado para la universidad: $91,440.00
Edad promedio por carrera:
Carrera
Contaduría                     29.000000
Psicología                     26.125000
Arquitectura                   25.500000
Derecho                        25.000000
Administración de Empresas     23.625000
Ingeniería Industrial          23.111111
Licenciatura en Matemáticas    22.400000
Ingeniería Electrónica         21.000000
Ingeniería de Sistemas         21.000000
Name: Edad, dtype: float64


In [133]:
# Verificar nombres de columnas
print(df.columns)

Index(['Nombre', 'Carrera', 'Costo_por_Credito', 'Costo_Materiales',
       'Fecha_Matricula', 'Pais_Procedencia', 'Costos_Administrativos',
       'Número de Cursos Matriculados', 'Incremento de Matrícula 2025 (%)',
       'Creditos_Matriculados', 'Promedio_Academico',
       'Número de Semestres Cursados', 'Edad', 'Costo_Total',
       'Costo_Proyeccion_2025'],
      dtype='object')


Si obtienes un error de KeyError, revisa que los nombres de las columnas coincidan exactamente con los usados en el cálculo de KPIs. Corrige los nombres si es necesario antes de continuar con los cálculos.

## 4. Visualización: Edad promedio por carrera (gráfico de barras)

Se muestra la edad promedio de los estudiantes agrupada por carrera.

In [134]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Gráfico de barras institucional: Edad promedio por carrera (mejorado)
fig = px.bar(
    edad_promedio_carrera.reset_index(),
    x='Carrera',
    y='Edad',
    title='Edad promedio por carrera',
    labels={'Edad': 'Edad promedio', 'Carrera': 'Carrera'},
    color='Edad',
    color_continuous_scale=[COLORES_CORHUILA['verde'], COLORES_CORHUILA['azul']]
)
fig.update_traces(marker_line_color=COLORES_CORHUILA['azul'], marker_line_width=2, marker=dict(line=dict(width=2), opacity=0.92))
fig.update_layout(xaxis_tickangle=-45)
fig = corhuila_layout(fig)
fig.show()

La gráfica anterior permite comparar visualmente la edad promedio de los estudiantes en cada carrera. ¿Deseas continuar con la siguiente visualización?

## 5. Visualización: Mapa de procedencia de estudiantes (por país)

Se muestra la distribución de estudiantes según su país de procedencia.

In [135]:
# Conteo de estudiantes por país
pais_counts = df['Pais_Procedencia'].value_counts().reset_index()
pais_counts.columns = ['Pais', 'Cantidad']

# Gráfico de mapa con Plotly (colores institucionales y logo, mejorado)
fig = px.choropleth(
    pais_counts,
    locations='Pais',
    locationmode='country names',
    color='Cantidad',
    title='Procedencia de estudiantes por país',
    color_continuous_scale=[COLORES_CORHUILA['verde'], COLORES_CORHUILA['azul']]
)
fig.update_layout(geo=dict(showframe=False, bgcolor=COLORES_CORHUILA['gris_claro']))
fig = corhuila_layout(fig)
fig.show()

Este mapa permite visualizar la diversidad geográfica de los estudiantes matriculados. ¿Continuamos con la siguiente visualización?

## 6. Visualización: Matrículas por fecha (gráfico de líneas)

Se muestra la tendencia temporal de matrículas a lo largo del tiempo.

In [136]:
# Agrupar y contar matrículas por fecha
matriculas_fecha = df.groupby('Fecha_Matricula').size().reset_index(name='Cantidad')

# Gráfico de líneas con Plotly (colores institucionales y logo, mejorado)
fig = px.line(
    matriculas_fecha,
    x='Fecha_Matricula',
    y='Cantidad',
    title='Tendencia de matrículas por fecha',
    labels={'Fecha_Matricula': 'Fecha de Matrícula', 'Cantidad': 'Cantidad de Matrículas'},
    color_discrete_sequence=[COLORES_CORHUILA['verde']]
)
fig.update_traces(line=dict(width=4), marker=dict(opacity=0.92))
fig.update_xaxes(dtick="M1", tickformat="%Y-%m-%d", tickangle=-45)
fig = corhuila_layout(fig)
fig.show()

Este gráfico permite analizar la evolución de las matrículas a lo largo del tiempo. ¿Deseas continuar con la siguiente visualización?

## 7. Visualización: Distribución de estudiantes por carrera (gráfico de barras y anillo)

Se muestra la cantidad de estudiantes matriculados en cada carrera, tanto en gráfico de barras como en gráfico de anillo.

In [137]:
# Conteo de estudiantes por carrera
distribucion_carrera = df['Carrera'].value_counts().reset_index()
distribucion_carrera.columns = ['Carrera', 'Cantidad']

# Gráfico de barras (colores institucionales y logo, mejorado)
fig_bar = px.bar(
    distribucion_carrera,
    x='Carrera',
    y='Cantidad',
    title='Distribución de estudiantes por carrera (Barras)',
    labels={'Cantidad': 'Cantidad de Estudiantes', 'Carrera': 'Carrera'},
    color='Cantidad',
    color_continuous_scale=[COLORES_CORHUILA['verde'], COLORES_CORHUILA['azul']]
)
fig_bar.update_traces(marker_line_color=COLORES_CORHUILA['azul'], marker_line_width=2, marker=dict(line=dict(width=2), opacity=0.92))
fig_bar.update_layout(xaxis_tickangle=-45)
fig_bar = corhuila_layout(fig_bar)
fig_bar.show()

# Gráfico de anillo (pie con hueco, colores institucionales y logo, mejorado)
fig_pie = px.pie(
    distribucion_carrera,
    names='Carrera',
    values='Cantidad',
    title='Distribución de estudiantes por carrera (Anillo)',
    hole=0.4,
    color_discrete_sequence=[COLORES_CORHUILA['verde'], COLORES_CORHUILA['azul']]
)
fig_pie.update_traces(textinfo='percent+label', pull=[0.03]*len(distribucion_carrera), marker=dict(line=dict(color=COLORES_CORHUILA['azul'], width=2)))
fig_pie = corhuila_layout(fig_pie)
fig_pie.show()

Estos gráficos permiten visualizar la proporción y cantidad de estudiantes en cada carrera, facilitando la comparación entre programas académicos.
¿Deseas continuar con las visualizaciones académicas (promedio académico, relaciones y dispersión)?

## 7. Visualización: Promedio académico por carrera (gráfico de columnas)

Se muestra el promedio académico de los estudiantes agrupado por carrera para identificar diferencias de rendimiento entre programas.

In [138]:
# Calcular promedio académico por carrera
promedio_academico_carrera = df.groupby('Carrera')['Promedio_Academico'].mean().sort_values(ascending=False).reset_index()

# Gráfico de columnas con Plotly (colores institucionales y logo, mejorado)
fig = px.bar(
    promedio_academico_carrera,
    x='Carrera',
    y='Promedio_Academico',
    title='Promedio académico por carrera',
    labels={'Promedio_Academico': 'Promedio Académico', 'Carrera': 'Carrera'},
    color='Promedio_Academico',
    color_continuous_scale=[COLORES_CORHUILA['verde'], COLORES_CORHUILA['azul']]
)
fig.update_traces(marker_line_color=COLORES_CORHUILA['azul'], marker_line_width=2, marker=dict(line=dict(width=2), opacity=0.92))
fig.update_layout(xaxis_tickangle=-45)
fig = corhuila_layout(fig)
fig.show()

El gráfico anterior permite comparar el rendimiento académico promedio entre las diferentes carreras. Esto ayuda a identificar programas con mejor desempeño y posibles áreas de mejora. ¿Deseas continuar con la siguiente visualización académica?

## 8. Relación entre semestres cursados y créditos aprobados

A continuación se analiza la relación entre el número de semestres cursados y la cantidad de créditos aprobados por los estudiantes. Esto permite identificar patrones de avance académico y posibles rezagos.

In [139]:
# Verificar nombres exactos de columnas para semestres y créditos
print(df.columns.tolist())

['Nombre', 'Carrera', 'Costo_por_Credito', 'Costo_Materiales', 'Fecha_Matricula', 'Pais_Procedencia', 'Costos_Administrativos', 'Número de Cursos Matriculados', 'Incremento de Matrícula 2025 (%)', 'Creditos_Matriculados', 'Promedio_Academico', 'Número de Semestres Cursados', 'Edad', 'Costo_Total', 'Costo_Proyeccion_2025']


El gráfico de dispersión permite observar si los estudiantes avanzan en créditos conforme cursan más semestres. Puntos alejados de la diagonal pueden indicar rezago o avance acelerado. ¿Deseas continuar con la siguiente visualización académica?

In [140]:
# Gráfico de dispersión: Semestres cursados vs. Créditos matriculados (colores institucionales y logo, mejorado)
fig = px.scatter(
    df,
    x='Número de Semestres Cursados',
    y='Creditos_Matriculados',
    color='Carrera',
    title='Relación entre semestres cursados y créditos matriculados',
    labels={'Número de Semestres Cursados': 'Semestres Cursados', 'Creditos_Matriculados': 'Créditos Matriculados'},
    hover_data=['Nombre'],
    color_discrete_sequence=[COLORES_CORHUILA['verde'], COLORES_CORHUILA['azul']]
)
fig.update_traces(marker=dict(size=13, opacity=0.75, line=dict(width=2, color=COLORES_CORHUILA['azul']), symbol='circle'))
fig = corhuila_layout(fig)
fig.show()

## 9. Relación entre créditos matriculados y promedio académico

Ahora analizaremos si existe una relación entre la cantidad de créditos matriculados y el promedio académico de los estudiantes. Esto puede ayudar a identificar si una mayor carga académica afecta el rendimiento.

In [141]:
# Gráfico de dispersión: Créditos matriculados vs. Promedio académico (colores institucionales y logo, mejorado)
fig = px.scatter(
    df,
    x='Creditos_Matriculados',
    y='Promedio_Academico',
    color='Carrera',
    title='Relación entre créditos matriculados y promedio académico',
    labels={'Creditos_Matriculados': 'Créditos Matriculados', 'Promedio_Academico': 'Promedio Académico'},
    hover_data=['Nombre'],
    color_discrete_sequence=[COLORES_CORHUILA['verde'], COLORES_CORHUILA['azul']]
)
fig.update_traces(marker=dict(size=13, opacity=0.75, line=dict(width=2, color=COLORES_CORHUILA['azul']), symbol='circle'))
fig = corhuila_layout(fig)
fig.show()

El gráfico permite observar si los estudiantes con mayor carga de créditos tienden a tener promedios académicos más altos o bajos. Analizar esta relación puede ser útil para orientar políticas de acompañamiento académico.

## 10. Relación entre edad y promedio académico

A continuación se analiza si existe alguna relación entre la edad de los estudiantes y su rendimiento académico promedio. Esto puede ayudar a identificar tendencias por grupo etario.

In [142]:
# Gráfico de dispersión: Edad vs. Promedio académico (colores institucionales y logo, mejorado)
fig = px.scatter(
    df,
    x='Edad',
    y='Promedio_Academico',
    color='Carrera',
    title='Relación entre edad y promedio académico',
    labels={'Edad': 'Edad', 'Promedio_Academico': 'Promedio Académico'},
    hover_data=['Nombre'],
    color_discrete_sequence=[COLORES_CORHUILA['verde'], COLORES_CORHUILA['azul']]
)
fig.update_traces(marker=dict(size=13, opacity=0.75, line=dict(width=2, color=COLORES_CORHUILA['azul']), symbol='circle'))
fig = corhuila_layout(fig)
fig.show()

Este gráfico permite observar si existen diferencias de rendimiento académico según la edad de los estudiantes. Puede ser útil para identificar necesidades de apoyo en ciertos grupos etarios.

Para agregar el logo de CORHUILA a los gráficos, puedes usar la opción `add_layout_image` de Plotly. Descarga el logo institucional en formato PNG y colócalo en la carpeta del notebook. Ejemplo:

Con esto, tus visualizaciones tendrán identidad institucional. Puedes aplicar la paleta y el logo a los gráficos principales del tablero para cumplir con los criterios de presentación de la actividad.

# Conclusiones y Recomendaciones Finales

El tablero interactivo desarrollado cumple con los lineamientos de identidad visual institucional de CORHUILA, utilizando exclusivamente la paleta de colores oficial y el logo institucional en todas las visualizaciones principales y secundarias. Esto garantiza coherencia, profesionalismo y refuerza la imagen de la universidad en la presentación de resultados.

**Conclusiones:**
- La aplicación de la paleta institucional y el logo en los gráficos facilita la identificación de la información como parte de CORHUILA y mejora la percepción de calidad.
- Las visualizaciones permiten analizar de manera clara y efectiva los principales indicadores académicos y demográficos de la matrícula universitaria.
- El tablero es fácilmente actualizable y escalable para futuras cohortes o variables adicionales.

**Recomendaciones:**
- Mantener la identidad visual en cualquier visualización futura, utilizando los bloques de código institucional proporcionados.
- Actualizar el logo institucional si la universidad realiza cambios en su imagen corporativa.
- Considerar la exportación del tablero a PDF institucional, incluyendo capturas de las visualizaciones, para su difusión en informes oficiales.
- Fomentar el uso de este tablero como herramienta de análisis y toma de decisiones en áreas académicas y administrativas.

---

*Este informe fue generado siguiendo los lineamientos de presentación institucional de la Universidad CORHUILA.*

# Referencias

Few, S. (2012). *Show me the numbers: Designing tables and graphs to enlighten* (2nd ed.). Analytics Press.

Gartner. (2023). *Magic quadrant for analytics and business intelligence platforms*. https://www.gartner.com/en/documents/4000175-magic-quadrant-for-analytics-and-business-intelligence-platforms

Knaflic, C. N. (2015). *Storytelling with data: A data visualization guide for business professionals*. Wiley.

McKinney, W. (2022). *Python for data analysis: Data wrangling with pandas, NumPy, and Jupyter* (3rd ed.). O’Reilly Media.

Microsoft. (n.d.). *Power BI*. https://powerbi.microsoft.com/

Microsoft. (n.d.). *Power BI customer stories*. https://powerbi.microsoft.com/en-us/customer-stories/

Murray, D. (2016). *Power BI: A comprehensive beginner’s guide to learn the basics and effective methods of Power BI*. CreateSpace Independent Publishing Platform.

Wexler, S., Shaffer, J., & Cotgreave, A. (2017). *The big book of dashboards: Visualizing your data using real-world business scenarios*. Wiley.